In [1]:
import xmltodict
import json
from tqdm import tqdm
import pandas as pd
import os

# xml 안에서 데이터 추출
- 원하는 데이터들만 json type 으로 변경하고 그에 대한 분포를 dataframe 형태로 만들자
- 필요없는 이미지 리스트를 만들어 놓는게 좋을까 고민은 해볼 것(data root 경로를 뺀 상태에서 이미지 리스트를 파일로 저장해두자)

In [2]:
XML_DIR = '/opt/ml/data/team_github/xml'
XML_LIST = [i for i in os.listdir(XML_DIR) if i.endswith('.xml')]
print(len(XML_LIST))
print(XML_LIST[0])

800
Surface_3-Surface_433-573_SM1022_IMT04.xml


In [12]:
# 전체 정보를 가져와서 data frame 형태로 만들어야 한다. -> 그래야 추후에 이미지 검증에도 활용 가능하다
# 그 다음에 필요한 클래스를 가지는 좌표를 가진 이미지만 쿼리를 통해서 빼낸다.
# 주석 처리한 print 는 검증용이니 확인해보셔도 좋을 것 같아요
def extract_xml_info(xml_file):
    with open(os.path.join(XML_DIR, xml_file), 'r') as f:
        #xml 모양의 str 이어야 동작된다.
        raw_xml = xmltodict.parse(f.read())
        #자주 사용한 json dict 로 변환하기 위함(ordereddict 가 반환되어 그렇슴)
        raw_xml = json.dumps(raw_xml)
        xml_to_json = json.loads(raw_xml)

    root_folder, sub_folder, xml_name = xml_file.split('-')
    # print(root_folder, sub_folder, xml_name)
    # print(f"#images : {len(xml_to_json['annotations']['image'])}")
    #for dataframe
    return_list = []
    # 이미지 별로
    # box_count = 0

    for image in xml_to_json['annotations']['image']:
        # segmetnation : dict_keys(['@id', '@name', '@width', '@height', 'polygon'])
        # {'@label': 'roadway', '@occluded': '0', '@points': '', '@z_order': '2', 'attribute': {'@name': 'attribute', '#text': 'normal'}}
        iserror = False
        temp_list = []
        
        if 'polygon' not in image.keys():
            print(root_folder, sub_folder, xml_name, image['@name'])
            continue

        if type(image['polygon']) != list:
            # extend 하는 경우에 리스트가 풀어져서 합쳐지기 때문에 dim=2 여야한다.

            attribute = image['polygon']['attribute']['#text'] if 'attribute' in image['polygon'].keys() else "None"

            temp_list = [[root_folder, sub_folder, xml_name, image['@name'],

                        image['polygon']['@label'], image['polygon']['@occluded'],
                        image['polygon']['@points'], attribute,
                        ]]
        else:
            for polygon in image['polygon']:
                
                # if type(polygon) != dict:
                #     print(root_folder, sub_folder, xml_name, image['@name'], polygon)

                try:
                    attribute = polygon['attribute']['#text'] if 'attribute' in polygon.keys() else "None"
                except Exception as ex:
                    iserror = True
                    print(root_folder, sub_folder, xml_name, image['@name'], polygon, ex)
                
                if iserror:
                    continue

                polygon_info = [root_folder, sub_folder, xml_name, image['@name'],
                                polygon['@occluded'], polygon['@label'],
                                polygon['@points'], attribute,
                            ]
                temp_list.append(polygon_info)

        # 모은 정보를 리턴할 리스트에 extend
        # box_count += len(temp_list)
        return_list.extend(temp_list)
    # print(f'before extracting #box : {box_count}\t after #box : {len(return_list)}')
    return return_list

In [13]:
all_info = []
from tqdm import tqdm
for x in tqdm(XML_LIST):
    all_info.extend(extract_xml_info(x))
print()
print(len(all_info))

 83%|████████▎ | 661/800 [00:07<00:01, 106.82it/s]

Surface_1 Surface_095 126_SM1001_IMT15.xml MP_SEL_SUR_006997.jpg
Surface_1 Surface_095 126_SM1001_IMT15.xml MP_SEL_SUR_006998.jpg
Surface_1 Surface_095 126_SM1001_IMT15.xml MP_SEL_SUR_007001.jpg


 86%|████████▌ | 685/800 [00:07<00:01, 110.64it/s]

Surface_5 Surface_700 705_SM1104_03.xml MP_SEL_SUR_036052.jpg {'@label': 'alley', '@occluded': '0', '@points': '1378.38,0.00;1319.78,209.95;1178.16,185.53;1164.70,238.10;1023.52,198.55;849.35,159.48;673.54,123.67;512.39,96.00;479.30,93.30;473.32,0.00', '@z_order': '1', 'attribute': [{'@name': 'attribute', '#text': 'blocks'}, {'@name': 'attribute', '#text': 'normal'}]} list indices must be integers or slices, not str


100%|██████████| 800/800 [00:08<00:00, 90.95it/s] 


181794


In [14]:
import pandas as pd

columns = ['root_folder', 'sub_folder', 'xml_name', 'file_name', 'occluded', 'classes', 'polygon', 'subclass']
target_list = ('crosswalk')

pandas_dict = pd.DataFrame(all_info, columns=columns)

pandas_dict.to_csv('crosswalk_temp.csv' ,header=True, index=False)

In [15]:
target_frame = pandas_dict[pandas_dict['subclass'] == 'crosswalk']
print(f'total data : {len(pandas_dict)}\t target data : {len(target_frame)}')

total data : 181794	 target data : 4887


In [16]:
target_frame.to_csv('crosswalk.csv', header=True, index=False)